In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

In [ ]:
def get_event_links(page_number):
    """
    Fetches event links from a specific page number on Eventbrite.
    Returns a set of URLs.
    """
    url = f"https://www.eventbrite.com/d/ny--new-york/all-events/?page={page_number}"
    
    # Rotate user agents or use a standard one to look like a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        # Check if we've been redirected to page 1 (common behavior when pages run out)
        # or if we hit a 404
        if response.status_code == 404:
            return set()
        if response.url != url and "page=1" not in response.url and page_number != 1:
            print(f"  [!] Redirected to {response.url} - assuming end of pages.")
            return set()

        soup = BeautifulSoup(response.text, 'html.parser')
        event_links = set()

        # Method: JSON-LD (Structured Data) extraction
        scripts = soup.find_all('script', type='application/ld+json')
        
        for script in scripts:
            try:
                data = json.loads(script.string)
                # Helper to extract url from a single item dict
                def extract_url(item):
                    if 'url' in item:
                        return item['url']
                    elif 'item' in item and 'url' in item['item']: # Nested Schema
                        return item['item']['url']
                    return None

                if isinstance(data, list):
                    for item in data:
                        url = extract_url(item)
                        if url: event_links.add(url)
                
                elif isinstance(data, dict):
                    if 'itemListElement' in data:
                        for item in data['itemListElement']:
                            url = extract_url(item)
                            if url: event_links.add(url)
            except (json.JSONDecodeError, TypeError):
                continue
        
        # Fallback: If JSON fails, look for 'a' tags with specific patterns
        # Note: Eventbrite changes classes often, so checking href for '/e/' is safer
        if not event_links:
            links = soup.find_all('a', href=True)
            for link in links:
                href = link['href']
                if '/e/' in href and 'eventbrite.com' in href:
                    clean_link = href.split('?')[0] # Remove tracking params
                    event_links.add(clean_link)
                    
        return event_links

    except requests.exceptions.RequestException as e:
        print(f"  [!] Error fetching page {page_number}: {e}")
        return set()

In [ ]:
all_events = set()
page = 1
max_safety_limit = 500  # Hard stop to prevent infinite loops if logic fails

print("Starting scraper...")

while page <= max_safety_limit:
    print(f"Scraping Page {page}...", end=" ")
    
    new_links = get_event_links(page)
    
    # Stop condition: If no links are returned, we have likely reached the end
    if not new_links:
        print("No events found. Reached end of results.")
        break
    
    # Update master list
    initial_count = len(all_events)
    all_events.update(new_links)
    new_count = len(all_events)
    
    print(f"Found {len(new_links)} links. (Total unique: {new_count})")
    
    # If we didn't add any new unique links, we might be seeing a 'no results' page 
    # that still has promoted links we've already seen.
    if new_count == initial_count and page > 1:
        print("  [!] No new unique links found. Stopping to avoid duplicate loops.")
        break

    page += 1
    
    # PAUSE: Random sleep to be polite and avoid IP bans
    sleep_time = random.uniform(2, 5) 
    time.sleep(sleep_time)

print("-" * 30)
print(f"Scraping complete. Found {len(all_events)} unique event links.")

In [17]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import concurrent.futures
from datetime import datetime, timedelta

In [20]:
# --- Configuration ---
START_DATE = "2025-12-12"  # YYYY-MM-DD
DAYS_TO_SCRAPE = 30        # How many days from start date

# Concurrency Controls
MAX_DAY_WORKERS = 5        # How many days to process at once (Outer Loop)
MAX_PAGE_WORKERS = 10       # How many pages to scrape at once per day (Inner Loop)
# NOTE: Total concurrent requests = DAY_WORKERS * PAGE_WORKERS (approx 50 here)

BASE_URL = "https://www.eventbrite.com/d/ny--new-york/all-events/"

USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
]

In [21]:
def get_dates(start_date_str, num_days):
    """Generates a list of date strings (YYYY-MM-DD)"""
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    return [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(num_days)]

In [22]:
def scrape_single_page(date, page_number):
    """
    Inner Worker: Scrapes a specific page for a specific date.
    """
    # Construct URL with date filters
    params = {
        'page': page_number,
        'start_date': date,
        'end_date': date
    }
    
    # Random sleep to prevent "thundering herd" on the server
    time.sleep(random.uniform(1, 3))
    
    headers = {
        'User-Agent': random.choice(USER_AGENTS),
        'Accept-Language': 'en-US,en;q=0.9',
    }

    try:
        response = requests.get(BASE_URL, params=params, headers=headers, timeout=10)
        
        # 404 or redirect usually means end of results
        if response.status_code == 404:
            return set()
        if "page=1" in response.url and page_number > 1:
            return set()

        soup = BeautifulSoup(response.text, 'html.parser')
        links = set()

        # 1. JSON-LD Extraction
        scripts = soup.find_all('script', type='application/ld+json')
        for script in scripts:
            try:
                data = json.loads(script.string)
                if isinstance(data, list):
                    for item in data:
                        if 'url' in item: links.add(item['url'])
                elif isinstance(data, dict):
                    if 'itemListElement' in data:
                        for item in data['itemListElement']:
                            if 'url' in item: links.add(item['url'])
                            elif 'item' in item and 'url' in item['item']:
                                links.add(item['item']['url'])
            except: continue

        # 2. Fallback Extraction
        if not links:
            for a in soup.find_all('a', href=True):
                if '/e/' in a['href'] and 'eventbrite.com' in a['href']:
                    links.add(a['href'].split('?')[0])

        return links

    except Exception as e:
        # print(f"  [!] Error {date} pg {page_number}: {e}")
        return set()

In [23]:
def manage_day_scrape(target_date):
    """
    Outer Worker: Manages the scraping for a single day.
    Spawns inner workers to handle pages in batches.
    """
    print(f"📅 Starting Day: {target_date}")
    day_links = set()
    current_page = 1
    keep_scraping = True
    
    # We use a ThreadPool for the pages within this day
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_PAGE_WORKERS) as page_executor:
        
        while keep_scraping:
            # Create a batch of pages (e.g., try pages 1, 2, 3 concurrently)
            # We batch to avoid queuing 50 pages for a day that has 0 events.
            batch_size = MAX_PAGE_WORKERS
            futures = {}
            
            for i in range(batch_size):
                pg = current_page + i
                futures[page_executor.submit(scrape_single_page, target_date, pg)] = pg
            
            batch_has_results = False
            
            for future in concurrent.futures.as_completed(futures):
                page_num = futures[future]
                try:
                    links = future.result()
                    if links:
                        day_links.update(links)
                        batch_has_results = True
                        # print(f"   -> {target_date} Page {page_num}: Found {len(links)} links")
                    else:
                        # If a page returns empty, we might have hit the end.
                        pass
                except Exception:
                    continue
            
            # Decision Logic:
            # If the entire batch returned 0 links, we assume the day is done.
            # (Or if we reached a safety limit like 50 pages)
            if not batch_has_results or current_page > 50:
                keep_scraping = False
            else:
                current_page += batch_size
                time.sleep(1) # Breath between batches

    print(f"✅ Finished {target_date}: {len(day_links)} total events.")
    return day_links

In [24]:
all_unique_links = set()
dates_to_scrape = get_dates(START_DATE, DAYS_TO_SCRAPE)

print(f"Starting Scrape for {len(dates_to_scrape)} days...")
print(f"Configuration: {MAX_DAY_WORKERS} Day-Workers x {MAX_PAGE_WORKERS} Page-Workers")

# Outer Pool: Manages different days
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_DAY_WORKERS) as day_executor:
    
    # Submit all date tasks
    future_to_date = {day_executor.submit(manage_day_scrape, date): date for date in dates_to_scrape}
    
    for future in concurrent.futures.as_completed(future_to_date):
        date = future_to_date[future]
        try:
            links = future.result()
            all_unique_links.update(links)
        except Exception as e:
            print(f"❌ Critical failure on {date}: {e}")

print("-" * 30)
print(f"SCRAPE COMPLETE.")
print(f"Total Unique Event Links: {len(all_unique_links)}")

# Save to file
with open('nyc_events_dated.txt', 'w') as f:
    for link in all_unique_links:
        f.write(f"{link}\n")

Starting Scrape for 30 days...
Configuration: 5 Day-Workers x 10 Page-Workers
📅 Starting Day: 2025-12-12
📅 Starting Day: 2025-12-13
📅 Starting Day: 2025-12-14
📅 Starting Day: 2025-12-15
📅 Starting Day: 2025-12-16
✅ Finished 2025-12-15: 304 total events.
📅 Starting Day: 2025-12-17
✅ Finished 2025-12-16: 455 total events.
📅 Starting Day: 2025-12-18
✅ Finished 2025-12-13: 700 total events.
📅 Starting Day: 2025-12-19
✅ Finished 2025-12-14: 708 total events.
📅 Starting Day: 2025-12-20
✅ Finished 2025-12-12: 700 total events.
📅 Starting Day: 2025-12-21
✅ Finished 2025-12-17: 525 total events.
📅 Starting Day: 2025-12-22
✅ Finished 2025-12-18: 570 total events.
📅 Starting Day: 2025-12-23
✅ Finished 2025-12-21: 693 total events.
📅 Starting Day: 2025-12-24
✅ Finished 2025-12-19: 705 total events.
📅 Starting Day: 2025-12-25
✅ Finished 2025-12-20: 706 total events.
📅 Starting Day: 2025-12-26
✅ Finished 2025-12-22: 188 total events.
📅 Starting Day: 2025-12-27
✅ Finished 2025-12-23: 305 total events

In [9]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import random
import concurrent.futures
from datetime import datetime

# --- Configuration ---
MAX_WORKERS = 10
INPUT_FILE = 'nyc_events_dated.txt'
OUTPUT_FILE = 'nyc_events_precision.json'

def extract_event_precision(url):
    time.sleep(random.uniform(0.5, 1.5))
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200: return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Data Containers
        event_info = {}
        categories = set()

        # --- STEP 1: Extract the Hydration JSON (The Truth) ---
        # This is the massive JSON object Eventbrite uses to build the page
        server_data_pattern = re.compile(r'window\.__SERVER_DATA__\s*=\s*({.*?});', re.DOTALL)
        script_content = soup.find('script', string=server_data_pattern)
        
        if script_content:
            match = server_data_pattern.search(script_content.string)
            if match:
                try:
                    full_data = json.loads(match.group(1))
                    
                    # Navigate to the event object (Handle standard variants)
                    event_obj = None
                    
                    # Path A: Standard
                    if 'event' in full_data: 
                        event_obj = full_data['event']
                    # Path B: Nested in event_data
                    elif 'event_data' in full_data and 'event' in full_data['event_data']:
                        event_obj = full_data['event_data']['event']
                    
                    if event_obj:
                        # 1. Get Title/Date/Location directly from this clean object
                        event_info['name'] = event_obj.get('name', {}).get('text')
                        event_info['startDate'] = event_obj.get('start', {}).get('local') # ISO String
                        
                        # Location usually nested
                        if 'venue' in event_obj and event_obj['venue']:
                            event_info['location'] = event_obj['venue'].get('name') or event_obj['venue'].get('address', {}).get('address_1')

                        # --- STEP 2: Surgical Category Extraction ---
                        
                        # A. The Primary Category (e.g. "Music")
                        # Path: event -> category -> name
                        if 'category' in event_obj and event_obj['category']:
                            cat_name = event_obj['category'].get('name')
                            # Filter out 'undefined' or system names
                            if cat_name and "Badge" not in cat_name:
                                categories.add(cat_name)

                        # B. The Format (e.g. "Party" or "Class")
                        # Path: event -> format -> name
                        if 'format' in event_obj and event_obj['format']:
                            fmt_name = event_obj['format'].get('name')
                            if fmt_name:
                                categories.add(fmt_name)

                        # C. The Tags (e.g. "Cultural", "Hip Hop")
                        # Path: event -> tags -> [ { "display_name": "..." } ]
                        if 'tags' in event_obj and isinstance(event_obj['tags'], list):
                            for tag in event_obj['tags']:
                                if 'display_name' in tag:
                                    categories.add(tag['display_name'])
                                elif 'name' in tag: # Some variants use 'name'
                                    categories.add(tag['name'])
                                    
                except json.JSONDecodeError:
                    pass # JSON parsing failed, fall back to next methods

        # --- STEP 3: Fallback Normalization ---
        # If the JSON extraction failed completely, we return partial data
        
        title = event_info.get('name')
        if not title:
             # Fallback to meta tag
             meta_title = soup.find('meta', property='og:title')
             title = meta_title['content'] if meta_title else "Unknown Title"

        timestamp = None
        start_str = event_info.get('startDate')
        if start_str:
            try:
                # Eventbrite internal JSON usually gives "2025-12-31T19:00:00" (Local time)
                # We assume local time for simplicity or parse timezone if available
                dt = datetime.fromisoformat(start_str)
                timestamp = int(dt.timestamp() * 1000)
            except: pass
            
        loc_name = event_info.get('location', "Online")
        
        # --- STEP 4: Breadcrumb Backup (If JSON Categories failed) ---
        if not categories:
            # Look specifically for the "Breadcrumbs" data-testid
            # This is safer than random links
            breadcrumbs = soup.find('div', attrs={'data-testid': 'breadcrumbs'})
            if breadcrumbs:
                for link in breadcrumbs.find_all('a'):
                    text = link.get_text(strip=True)
                    if text not in ["Home", "United States", "New York", "Events", "NY"]:
                        categories.add(text)

        return {
            "Title": title,
            "DateTime": timestamp,
            "Location": loc_name,
            "Categories": list(categories),
            "Link": url
        }

    except Exception as e:
        # print(f"Error: {e}")
        return None

# --- Main Execution ---
if __name__ == "__main__":
    # Test on the specific link that was giving "Badge"
    test_links = [
        "https://www.eventbrite.com/e/african-dance-tickets-926482633497",
        "https://www.eventbrite.com/e/new-year-eve-hottest-bollywood-desi-party-racket-nyc-tickets-1977187088807"
    ]
    
    print("Running Precision Scraper...")
    for link in test_links:
        result = extract_event_precision(link)
        print(json.dumps(result, indent=4))

Running Precision Scraper...
null
null
